In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
from matplotlib import pyplot
pyplot.ion()
pyplot.style.use('dark_background')

import torch as th
from rl.algs.dqn import *

In [7]:
# GridWorld (debugging)
from rl.algs import experiment
from rl.algs import model
from rl.algs.envs import grid_world

BATCH_SIZE = 32
TRAIN_START_STEP = 3 * BATCH_SIZE
NUM_TRAIN_STEPS = 10000 # 17
REPLAY_SIZE = 100000
TARGET_NETWORK_UPDATE_FREQ = 500


env = environment.GridWorldWrapped(grid_shape=(4, 4, 1))
conv_specs = {
  model.ConvSpec(4, 1, 32),
}
fc_specs = (64,)
model_params = {
  'conv_specs': conv_specs,
  'fc_specs': fc_specs,
}

EPS_DECAY = util.Schedule((0, TRAIN_START_STEP, NUM_TRAIN_STEPS / 1, NUM_TRAIN_STEPS),
                          (1, 1, 0.0, 0.0))
EPS_RANDOM = util.Schedule((0,), (1.0,))
adam_params = {
  'fn': th.optim.Adam,
  'kwargs': {
    'lr': 0.005,
  }
}
rmsprop_params = {
  'fn': th.optim.RMSprop,
  'kwargs': {
    'lr': 0.00025,
    'momentum': 0.95,
    'alpha': 0.95,
    'eps': 0.01,
  }
}

policy_params = {
    'opt_params': rmsprop_params,
    'eps_sched': EPS_DECAY,  # TODO: change
    'target_update_freq': TARGET_NETWORK_UPDATE_FREQ,
}
exp = experiment.Experiment(env, model.QNetwork, model_params,
                            Dqn, policy_params)

print(exp.policy.model)

rb = ReplayBuffer(REPLAY_SIZE)
for step in range(TRAIN_START_STEP):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)

eps_rew = 0
for step in range(TRAIN_START_STEP, NUM_TRAIN_STEPS):
  sars = env.step(exp.policy.get_action)
  rb.add(sars)
    
  experience_batch = rb.sample(BATCH_SIZE)
  m = exp.policy.update(experience_batch)
  
  eps_rew += sars.r
  if sars.s1 is None:
    m['r_per_eps'] = eps_rew
    eps_rew = 0
  if step % 100 == 0 or sars.s1 is None:
    for k, v in m.items():
      exp.plt.add_data(k, step, v)
      
# Expect about r_per_eps=-50 in pure random agent.

(1, 1, 32)
QNetwork(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(4, 4), stride=(1, 1))
    (1): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=32, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [11]:
opt = grid_world.GridWorld((4, 4)).optimal_expected_reward()
print(f'optimal expected r: {opt}')

optimal expected r: 1.6666666666666665


In [8]:
env.show_model_policy(exp.policy.model)

[[ 0.          0.0313592  -1.03494978 -2.08643818]
 [-0.01728863 -0.95099533 -1.95232618 -3.0272541 ]
 [-1.04983127 -2.10587645 -2.9338026  -3.70709276]
 [-2.16567278 -3.00610065 -3.70971012 -4.5428772 ]]
[['⟲' '←' '←' '←']
 ['↑' '↑' '←' '↑']
 ['↑' '↑' '←' '←']
 ['↑' '←' '↑' '←']]
[[ 6.56600595e-02  0.00000000e+00  4.43780422e-03 -9.80988801e-01]
 [-9.34150040e-01 -7.84754753e-04 -9.77480114e-01 -2.03743219e+00]
 [-2.08350277e+00 -9.96392787e-01 -1.94335055e+00 -3.12432981e+00]
 [-2.97975206e+00 -2.01479077e+00 -2.96587849e+00 -3.89123440e+00]]
[['→' '⟲' '←' '←']
 ['↑' '↑' '↑' '↑']
 ['↑' '↑' '←' '↑']
 ['↑' '↑' '←' '←']]
[[-0.94160652  0.04264966  0.          0.00849825]
 [-1.87593126 -0.94237375  0.10772246 -0.81526321]
 [-2.7130909  -1.89360595 -0.8548336  -1.71022177]
 [-3.61952829 -2.74878621 -1.90353477 -2.81757545]]
[['→' '→' '⟲' '←']
 ['↑' '→' '↑' '↑']
 ['→' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑']]
[[-1.7695632  -0.85337484  0.07786766  0.        ]
 [-2.63022304 -1.79655099 -0.79483569  

In [9]:
# Debug the action-value function for a given state.
s = [[0, 0, 0, 0],
     [0, 0, 0, 0],
     [0, 0, 1, 0],
     [0, 0, 2, 0]]
s = np.expand_dims(np.expand_dims(s, 0), -1)
print(exp.policy.model(util.to_variable(s)))
print(exp.policy.target_model(util.to_variable(s)))

Variable containing:
-1.8820 -1.8912 -0.0071 -1.8693
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]

Variable containing:
-1.7349 -1.9211 -0.0394 -1.7967
[torch.cuda.FloatTensor of size 1x4 (GPU 0)]



In [10]:
# Examine the replay buffer.
terminals = rb.filter_by(sars_filter(r=0))
print(len(terminals))

# action-value at 's' was wrong. Search the replay buffer for occurrences of
# the correct transition. It's likely missing, which led the NN to overfit
# on missing data.
trans_s = rb.filter_by(sars_filter(s=s))
trans_s_a = rb.filter_by(sars_filter(s=s,a=2))
print(f'Num with (s): {len(trans_s)}. Num with (s, a): {len(trans_s_a)}')


1643
Num with (s): 74. Num with (s, a): 51


In [35]:
env.visualize(exp.policy, steps=20)

array([[0, 0],
       [1, 0]])
array([[0, 0],
       [1, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
array([[1, 0],
       [0, 0]])
